# Rapport TP1 Métaheuristiques d'optimization 

### Introduction:

Ce TP consiste à comparer la performance et l'efficacité de trois métaheuristiques du type Monte-Carlo. 
J'ai essayé de regrouper tout le code dans un seul notebook même si le développement de chaque algorithme est fait dans un fichier séparé accompagnant ce rapport. 
Ce notebook représente le rapport du travail et comporte tout le code source et le résultat de l'exécution avec des commentaires et des explications.
Pour les deux algorithmes du hill climbing et du recuit simulé, il se peut que l'algorithme rentre dans une boucle de très longue durée lors de la génération du nouveau candidat à cause des contraintes, il serait utile donc d'interrompre le Kernel et réexécuter l'appel de l'algorithme.

Deux bibliothèques seulement sont utilisées pour ce travail :

In [87]:
import numpy as np
import pandas as pd

La fonction 'bornes' permet de retourner les bornes des variables du problème et parce que les bornes des variables x3 et x4 dépendent des valeurs de z1 et z2, on doit calculer les bornes à chaque fois en fonction du nouveau point qu'on a.

Ces bornes n'étaient pas explicites, il fallait donc simplifier les différentes inégalités qu'on a dans le problème. Dans ce qui suit la démonstration :

•	x1 <= 99 : c'est une donnée, mais j'ai choisi comme borne inferieure 20 après plusieurs exécutions ou l'algorithme tournait dans une boucle très longue pour trouver un candidat acceptable.

•	x2 >= 1 : donnée, et j'ai choisi comme borne supérieure 90 pour la même raison

•	0.00954*x3 <= z2 & 0.0193*x3 <= z1 & x3 <= 200 ce qui implique que x3 <= min(0.625*x[1]/0.00954,0.625*x[0]/0.0193, 200), et j'ai choisi comme borne inferieure 1

•	-pi*x3**2 - 4/3*pi*x3**3 <= -1296000  & x3 >=10 implique que x3 >= 1296000/pi*x3**2 - 4/3*x3 (x3!=0) & x3 >= 10

ce qui implique que : x3 >= max(1296000/pi*x3**2 - 4/3*x3, 10) et on a x3 <= 240


In [88]:
def bornes(x):
    return np.asarray([[20,99],[1,90],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])

La fonction objective :

In [89]:
def cout_soudure(x):
    return 1.7781*0.625*x[1]*x[2]**2+ 0.6224*0.625*x[0]*x[2]*x[3] + 3.1661*(0.625*x[0])**2*x[3] + 19.84*(0.625*x[0])**2*x[2]


Cette fonction permet de vérifier si le nouveau point génère respecte les contraintes ou pas en se servant de la fonction 'bornes'. 

In [90]:
def contraintes(x):
    _bornes = bornes(x)
    for i in range(len(x)) :
        if x[i] < _bornes[i,0] or x[i] > _bornes[i,1]:
            return False
    return True


La fonction d'initialisation du premier point pour les deux algorithmes du hill climbing et du recuit simulé et qui le génère en respectant les contraintes imposées en se servant de la fonction 'bornes'. 

In [91]:
def initialiser_points():
    x = np.zeros(4)
    _bornes = bornes(x)
    x[0]= np.random.uniform(high = _bornes[0,1])
    x[1]= np.random.uniform(low =  _bornes[1,0], high =  _bornes[1,1])
    _bornes = bornes(x)
    x[2]= np.random.uniform(high =  _bornes[2,1])
    _bornes = bornes(x)
    x[3]= np.random.uniform(low =  _bornes[3,0], high =  _bornes[3,1])

    return x



Cette fonction est pour générer une liste de n points pour l'algorithme aléatoire.

In [92]:
def generer_points(n):
    x = np.zeros([n,4])
    for i in range(n):
        _bornes = bornes(x[i])
        x[i,0]= np.random.uniform(high = _bornes[0,1])
        x[i,1]= np.random.uniform(low =  _bornes[1,0], high =  _bornes[1,1])
        _bornes = bornes(x[i])
        x[i,2]= np.random.uniform(high =  _bornes[2,1])
        _bornes = bornes(x[i])
        x[i,3]= np.random.uniform(low =  _bornes[3,0], high =  _bornes[3,1])

    return x
   


La fonction 'mutation' est utilisée aussi pour les deux algorithmes du hill climbing et du recuit simulé. Elle permet de modifier une des variables choisies aléatoirement du candidat actuel. Elle retourne un nouveau point en fonction de l'ancien à condition qu'il respecte les contraintes.
Pour ce, j'ai utilisé une mutation trouvée dans une référence du cours qui se sert de la distribution gaussienne mais j'ai modifié la constante de réduction (0.01 ==> 0.1) parce que l'algorithme prenait un temps infini pour trouver un nouveau candidat acceptable, et l'augmentation de cette constante permet plus d'excitation du point et ne pas rester coince dans un petit intervalle. En plus, si la fonction ne trouve pas un nouveau candidat acceptable pendant dix itérations, elle change la variable à modifier.


In [93]:
def mutation(candidat):
    _bornes = bornes(candidat)

    
    temp_array = candidat

    cpt=0
    while True: #do-while
        
        if(cpt%10 == 0):
            i = np.random.randint(0,4)
            temp=candidat[i]
        
        temp = candidat[i] + np.random.normal(0,1) * 0.1 * temp * (_bornes[i,1]- _bornes[i,0]) 
        temp_array[i] = temp

        if contraintes(temp_array):
            candidat[i] = temp
            break
        
        cpt+=1
    return candidat


Les différentes heuristiques de refroidissement pour le recuit simulé:

In [94]:
def calculer_temperature_expo(t, T_init,epsilon_t=0.025): #exponentiel
    return (1-epsilon_t)**t*T_init



In [95]:
def calculer_temperature_log(t, T_init): # logarithmique
    if t<3:
        return T_init
    return T_init/np.log(t)


In [96]:
def calculer_temperature_poly(t,T_init,t_max=200, alpha=2): # polynomial
    return (1-t/t_max)**alpha * T_init # t_max est l'iteration maximale pour laquelle la temperature doit etre nulle


Fonction de calcul de la probalbilite d'acceptation d'un nouveau point dont le score est plus mauvais que celui du point atuel.

In [97]:
def proba_acceptance(T,deltaE):
    return np.exp(-deltaE/T)

'print_results' permet d'afficher les meilleurs résultats trouves. 

In [98]:
def print_results(hist):
    
    scores = hist[:,1]
    best_score = min(scores)
    index = np.where(scores == best_score)
    #print(f"index : {index}")
    print(f"meilleurs dimensions : \n z1 = {0.625*hist[index[0][0],3]} \n z2 = {0.625*hist[index[0][0],4]} \n x3 = {hist[index[0][0],5]} \n x4 = {hist[index[0][0],6]}\n")
    print(f"meilleur score : {best_score}\n")

   



Cette fonction prepare les donnees pour la presentation dans un tableau a l'aide de la bibliotheque 'pandas'. Elle permet de convertir une liste numpy en une DataFrame, avec le calcul de z1 et z2 a partir de x1 et x2.

In [99]:
def preparation_table(hist):
    hist[:,3] = [0.625 * a for a in hist[:,3]]
    hist[:,4] = [0.625 * a for a in hist[:,4]]

    table = pd.DataFrame(data=hist[:,1:], columns=["Score","Nbr itrs", "z1","z2","x3","x4"], index=hist[:,0]) 
    
    return table




Fonction pour l'affichage de l'historique détaillé des points et scores.

In [100]:
def print_table(hist):
      print("historique detaille :\n")
      return preparation_table(hist)

Fonction pour l'affichage d'une table contenant l'analyse de l'historique à l'aide de la méthode describe() de 'pandas'.

In [101]:
def print_analyse(hist):
    print("Analyse des donnees generees :\n")
    return preparation_table(hist).describe()


L'algorithme de la recherche aléatoire : 

In [154]:
def aleatoire(n = 50, itrs=100,epsilon = 0.001, samples_size = 10 ,verbose = False):
    
     
       #iterateur 
    i=0
    historique = np.empty([0,7])
    while(i<n):
        i+=1
        k=0
        l=0
        current_points = generer_points(samples_size)
        scores = np.apply_along_axis(cout_soudure, arr = current_points, axis = 1)
        score = np.min(scores)
        current_point = current_points[np.argmin(scores),]
        best_point = current_point
        best_score = score
        best_before_score = best_score
        
        while(k<itrs):
          k+=1   
          current_points = generer_points(samples_size)
          scores = np.apply_along_axis(cout_soudure, arr = current_points, axis = 1)

          score = np.min(scores)
          current_point = current_points[np.argmin(scores),]          
          
          best_before_score = best_score

          if(score < best_score):
                best_score = score
                best_point = current_point
          

          if best_before_score - best_score < epsilon :
                 l += 1 
          else :
                 l = 0

          if l == 50 : 
              break
          

          if verbose :
                print(f"itr : {k}, minimum : {current_point}, best score : {score}")
        
        historique = np.append(historique,[np.append([i,best_score,k],[best_point])],axis=0)
    return historique

In [155]:
while True:
    iter = input("Entrer le nombre d'iterations : ")
    epsilon = input("Entrer epsilon ( progres minimal ) : ")

    if int(iter) > 0 and float(epsilon) > 0:
      break
    else:
      print("Veuillez reesayer!")
    
hist = aleatoire(itrs = int(iter), epsilon=float(epsilon))

Entrer le nombre d'iterations :  10
Entrer epsilon ( progres minimal ) :  0.01


<ipython-input-88-d5d27ed1eefe>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.asarray([[20,99],[1,90],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])


In [156]:
print_results(hist)

meilleurs dimensions : 
 z1 = 0.10967516169994439 
 z2 = 1.3900929424471427 
 x3 = 3.9140281888200943 
 x4 = 6508.165654562978

meilleur score : 2025.4993228878886



In [157]:
cout_soudure([0.10967516169994439 /0.625 ,1.3900929424471427 /0.625,3.9140281888200943   , 6508.165654562978])

2025.4993228878886

In [147]:
print_table(hist)

historique detaille :



,Score,Nbr itrs,z1,z2,x3,x4
1.0,8739.101470,10.0,0.348447,21.177219,11.655083,1.234819e+03
2.0,1142.589584,10.0,0.449237,50.646601,2.699785,3.410628e+02
3.0,15617.474135,10.0,1.206392,3.185345,33.276868,2.833247e+02
4.0,105101.333874,10.0,7.300744,1.309975,65.237642,5.632049e+01
5.0,5333.044134,10.0,0.334448,52.265618,4.108778,3.104776e+03
6.0,43359.226424,10.0,2.702651,5.679444,50.253161,9.821608e+01
7.0,27172.174104,10.0,1.252451,11.315218,24.010052,6.260362e+02
8.0,4446.410424,10.0,0.292373,25.130292,8.862704,4.892963e+02
9.0,26890.823583,10.0,1.308901,46.087446,10.477231,1.256415e+03
10.0,118135.358169,10.0,2.283404,34.624186,40.046545,2.078266e+02


In [148]:
print_analyse(hist)

Analyse des donnees generees :



,Score,Nbr itrs,z1,z2,x3,x4
count,50.000000,50.0,50.000000,50.000000,50.000000,5.000000e+01
mean,28096.425604,10.0,0.711670,15.485013,17.447148,3.995595e+04
std,33351.418918,0.0,0.864776,10.166518,16.313975,2.365565e+05
min,1142.589584,10.0,0.006160,0.818735,0.494226,5.632049e+01
25%,6011.085953,10.0,0.202253,7.159970,4.435367,3.089582e+02
50%,15755.079165,10.0,0.433400,13.469474,10.764824,1.092605e+03
75%,32787.249303,10.0,0.809243,24.167412,27.305750,3.107985e+03
max,150478.909710,10.0,4.562965,33.554027,65.237642,1.676078e+06


L'algorithme du Hill Climbing : 

In [107]:
def hill_climbing(itrs = 100,n = 50, epsilon = 0.001, verbose = False):
    i=0
    historique = np.empty([0,7])

    while(i<n):
      i+=1
      currentPoint = initialiser_points()
      bestScore = cout_soudure(currentPoint)
      bestPoint = currentPoint
      beforeBestScore = bestScore
      k=0 #iterateur 
      l=0

      while(k<itrs):

          k+=1   
          currentPoint = mutation(currentPoint)
          score = cout_soudure(currentPoint)
          
          beforeBestScore = bestScore

          if(score < bestScore):
                bestScore = score
                bestPoint = currentPoint

          if beforeBestScore - bestScore < epsilon :
                 l += 1 
          else :
                 l = 0

          if l == 50 : 
              break
          

          if verbose :
                print(f"itr : {k}, minimum : {currentPoint},  score : {score}")
    
      historique = np.append(historique,[np.append([i,bestScore,k],[bestPoint])],axis=0)

    return historique

In [109]:
while True:
    iter = input("Entrer le nombre d'iterations : ")
    epsilon = input("Entrer epsilon ( progres minimal ) : ")

    if int(iter) > 0 and float(epsilon) > 0:
      break
    else:
      print("Veuillez reesayer!")
    
hist = hill_climbing(itrs = int(iter), epsilon=float(epsilon))

Entrer le nombre d'iterations :  10
Entrer epsilon ( progres minimal ) :  0.01


<ipython-input-88-d5d27ed1eefe>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.asarray([[20,99],[1,90],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])
<ipython-input-88-d5d27ed1eefe>:2: RuntimeWarning: overflow encountered in double_scalars
  return np.asarray([[20,99],[1,90],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])
<ipython-input-93-01ceb51c518d>:14: RuntimeWarning: overflow encountered in double_scalars
  temp = candidat[i] + np.random.normal(0,1) * 0.1 * temp * (_bornes[i,1]- _bornes[i,0])
<ipython-input-93-01ceb51c518d>:14: RuntimeWarning: invalid value encountered in double_scalars
  temp = candidat[i] + np.random.normal(0,1) * 0.1 * temp * (_bornes[i,1]- _bornes[i,0])


In [110]:
print_results(hist)

meilleurs dimensions : 
 z1 = nan 
 z2 = nan 
 x3 = nan 
 x4 = nan

meilleur score : 11472.343330839867



In [111]:
print_table(hist)

historique detaille :



,Score,Nbr itrs,z1,z2,x3,x4
1.0,2.071193e+06,10.0,NaN,NaN,NaN,NaN
2.0,8.976210e+05,10.0,NaN,NaN,NaN,NaN
3.0,2.905953e+05,10.0,NaN,NaN,NaN,NaN
4.0,1.253217e+06,10.0,NaN,NaN,NaN,NaN
5.0,1.736549e+06,10.0,NaN,NaN,NaN,NaN
6.0,5.777901e+05,10.0,NaN,NaN,NaN,NaN
7.0,4.031697e+06,10.0,NaN,NaN,NaN,NaN
8.0,3.515628e+06,10.0,NaN,NaN,NaN,NaN
9.0,1.034275e+07,10.0,NaN,NaN,NaN,NaN
10.0,6.935583e+06,10.0,NaN,NaN,NaN,NaN


In [112]:
print_analyse(hist)

Analyse des donnees generees :



,Score,Nbr itrs,z1,z2,x3,x4
count,5.000000e+01,50.0,0.0,0.0,0.0,0.0
mean,1.517861e+07,10.0,NaN,NaN,NaN,NaN
std,7.549316e+07,0.0,NaN,NaN,NaN,NaN
min,1.147234e+04,10.0,NaN,NaN,NaN,NaN
25%,1.429441e+06,10.0,NaN,NaN,NaN,NaN
50%,3.617789e+06,10.0,NaN,NaN,NaN,NaN
75%,6.356182e+06,10.0,NaN,NaN,NaN,NaN
max,5.377022e+08,10.0,NaN,NaN,NaN,NaN


L'algorithme du recuit simulé : 
Après chaque m itérations, on ajoute un nombre aléatoire par la loi uniforme entre 0 et 1. Par défaut, m est fixe à 10 itérations.


In [113]:
def recuit_simule(T_init = 400,temperature= calculer_temperature_poly,itrs= 100,n = 50, m=10,epsilon = 0.001, verbose = False):
      # T_init ne doit pas etre posee comme ça
      
    
    historique = np.empty([0,7])
    i=0
    while(i<n):
        i+=1
        current_point = initialiser_points()
        best_point = current_point
        current_score = cout_soudure(current_point)
        best_score = current_score
        bestc_before_score = best_score
        k=0 # iterateur 
        l=0 # iterations pour suivre l'amelioration
        while(k<itrs):
          k+= 1
          
          new_point = mutation(current_point)
          new_score = cout_soudure(new_point)
          
          deltaE = new_score - current_score
          
          best_before_score = best_score

          if  deltaE <= 0 : 
              current_point = new_point
              current_score = new_score
              if current_score < best_score:
                  best_point = current_point
                  best_score = current_score
                
          else :
              T = temperature(k,T_init) + k//m * np.random.uniform(0,1) # Rechauffement apres chaque m iterations
              if np.random.uniform(0,1) < proba_acceptance(T,deltaE):
                  current_point = new_point
                  current_score = new_score


          if best_before_score - best_score < epsilon :  #peu ou pas d'amelioration 
                 l += 1 
          else :
                 l = 0

          if l == 50 : 
              current_point = best_point         #Retour au dernier meilleur point
              current_score = best_score
          

          if verbose :
                print(f"itr : {k}, point : {current_point}, score : {current_score}")
                
        historique = np.append(historique,[np.append([i,best_score,k],[best_point])],axis=0)

    return historique

Resultats pour le refroidissement polynomial : 

In [114]:
while True:
    iter = input("Entrer le nombre d'iterations : ")
    epsilon = input("Entrer epsilon ( progres minimal ) : ")

    if int(iter) > 0 and float(epsilon) > 0:
      break
    else:
      print("Une des valeurs entrees n'est pas acceptee!")
    
hist = recuit_simule(itrs = int(iter), epsilon=float(epsilon))

Entrer le nombre d'iterations :  10
Entrer epsilon ( progres minimal ) :  0.01


<ipython-input-88-d5d27ed1eefe>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.asarray([[20,99],[1,90],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])
<ipython-input-93-01ceb51c518d>:14: RuntimeWarning: overflow encountered in double_scalars
  temp = candidat[i] + np.random.normal(0,1) * 0.1 * temp * (_bornes[i,1]- _bornes[i,0])
<ipython-input-93-01ceb51c518d>:14: RuntimeWarning: invalid value encountered in double_scalars
  temp = candidat[i] + np.random.normal(0,1) * 0.1 * temp * (_bornes[i,1]- _bornes[i,0])
<ipython-input-88-d5d27ed1eefe>:2: RuntimeWarning: overflow encountered in double_scalars
  return np.asarray([[20,99],[1,90],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])


In [115]:
print_results(hist)

meilleurs dimensions : 
 z1 = nan 
 z2 = nan 
 x3 = nan 
 x4 = nan

meilleur score : 74600.05620294383



In [116]:
print_table(hist)

historique detaille :



,Score,Nbr itrs,z1,z2,x3,x4
1.0,1.428916e+08,10.0,NaN,NaN,NaN,NaN
2.0,4.869368e+06,10.0,NaN,NaN,NaN,NaN
3.0,2.848856e+06,10.0,NaN,NaN,NaN,NaN
4.0,1.046325e+06,10.0,NaN,NaN,NaN,NaN
5.0,9.022828e+06,10.0,NaN,NaN,NaN,NaN
6.0,1.252210e+05,10.0,NaN,NaN,NaN,NaN
7.0,6.521440e+06,10.0,NaN,NaN,NaN,NaN
8.0,4.473460e+06,10.0,NaN,NaN,NaN,NaN
9.0,2.087612e+06,10.0,NaN,NaN,NaN,NaN
10.0,5.878205e+05,10.0,NaN,NaN,NaN,NaN


In [117]:
print_analyse(hist)

Analyse des donnees generees :



,Score,Nbr itrs,z1,z2,x3,x4
count,5.000000e+01,50.0,0.0,0.0,0.0,0.0
mean,9.891300e+06,10.0,NaN,NaN,NaN,NaN
std,2.767697e+07,0.0,NaN,NaN,NaN,NaN
min,7.460006e+04,10.0,NaN,NaN,NaN,NaN
25%,1.439294e+06,10.0,NaN,NaN,NaN,NaN
50%,2.978032e+06,10.0,NaN,NaN,NaN,NaN
75%,5.259126e+06,10.0,NaN,NaN,NaN,NaN
max,1.428916e+08,10.0,NaN,NaN,NaN,NaN


Resultats pour le refroidissement exponentiel : 

In [132]:
while True:
    iter = input("Entrer le nombre d'iterations : ")
    epsilon = input("Entrer epsilon ( progres minimal ) : ")

    if int(iter) > 0 and float(epsilon) > 0:
      break
    else:
      print("Une des valeurs entrees n'est pas acceptee!")
    
hist = recuit_simule(temperature = calculer_temperature_expo,itrs = int(iter), epsilon=float(epsilon))

Entrer le nombre d'iterations :  10
Entrer epsilon ( progres minimal ) :  0.01


<ipython-input-88-d5d27ed1eefe>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.asarray([[20,99],[1,90],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])
<ipython-input-88-d5d27ed1eefe>:2: RuntimeWarning: overflow encountered in double_scalars
  return np.asarray([[20,99],[1,90],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])
<ipython-input-93-01ceb51c518d>:14: RuntimeWarning: overflow encountered in double_scalars
  temp = candidat[i] + np.random.normal(0,1) * 0.1 * temp * (_bornes[i,1]- _bornes[i,0])
<ipython-input-93-01ceb51c518d>:14: RuntimeWarning: invalid value encountered in double_scalars
  temp = candidat[i] + np.random.normal(0,1) * 0.1 * temp * (_bornes[i,1]- _bornes[i,0])


In [133]:
print_results(hist)

meilleurs dimensions : 
 z1 = nan 
 z2 = nan 
 x3 = nan 
 x4 = nan

meilleur score : 80722.41046934108



In [134]:
print_table(hist)

historique detaille :



,Score,Nbr itrs,z1,z2,x3,x4
1.0,6.842398e+06,10.0,NaN,NaN,NaN,NaN
2.0,3.953953e+06,10.0,NaN,NaN,NaN,NaN
3.0,4.497081e+06,10.0,NaN,NaN,NaN,NaN
4.0,7.833593e+06,10.0,NaN,NaN,NaN,NaN
5.0,2.681470e+06,10.0,NaN,NaN,NaN,NaN
6.0,3.202396e+05,10.0,NaN,NaN,NaN,NaN
7.0,2.616199e+06,10.0,NaN,NaN,NaN,NaN
8.0,1.831235e+06,10.0,NaN,NaN,NaN,NaN
9.0,1.671030e+06,10.0,NaN,NaN,NaN,NaN
10.0,1.769768e+06,10.0,NaN,NaN,NaN,NaN


In [135]:
print_analyse(hist)

Analyse des donnees generees :



,Score,Nbr itrs,z1,z2,x3,x4
count,5.000000e+01,50.0,0.0,0.0,0.0,0.0
mean,4.138529e+06,10.0,NaN,NaN,NaN,NaN
std,4.137312e+06,0.0,NaN,NaN,NaN,NaN
min,8.072241e+04,10.0,NaN,NaN,NaN,NaN
25%,1.552045e+06,10.0,NaN,NaN,NaN,NaN
50%,2.718614e+06,10.0,NaN,NaN,NaN,NaN
75%,5.733900e+06,10.0,NaN,NaN,NaN,NaN
max,1.781023e+07,10.0,NaN,NaN,NaN,NaN


Resultats pour le refroidissement logarithmique : 

In [140]:
while True:
    iter = input("Entrer le nombre d'iterations : ")
    epsilon = input("Entrer epsilon ( progres minimal ) : ")

    if int(iter) > 0 and float(epsilon) > 0:
      break
    else:
      print("Une des valeurs entrees n'est pas acceptee!")
    
hist = recuit_simule(temperature = calculer_temperature_log,itrs = int(iter), epsilon=float(epsilon))

Entrer le nombre d'iterations :  10
Entrer epsilon ( progres minimal ) :  0.01


<ipython-input-88-d5d27ed1eefe>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.asarray([[20,99],[1,90],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])
<ipython-input-88-d5d27ed1eefe>:2: RuntimeWarning: overflow encountered in double_scalars
  return np.asarray([[20,99],[1,90],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])
<ipython-input-93-01ceb51c518d>:14: RuntimeWarning: overflow encountered in double_scalars
  temp = candidat[i] + np.random.normal(0,1) * 0.1 * temp * (_bornes[i,1]- _bornes[i,0])
<ipython-input-93-01ceb51c518d>:14: RuntimeWarning: invalid value encountered in double_scalars
  temp = candidat[i] + np.random.normal(0,1) * 0.1 * temp * (_bornes[i,1]- _bornes[i,0])


In [141]:
print_results(hist)

meilleurs dimensions : 
 z1 = nan 
 z2 = nan 
 x3 = nan 
 x4 = nan

meilleur score : 86808.22291831818



In [142]:
print_table(hist)

historique detaille :



,Score,Nbr itrs,z1,z2,x3,x4
1.0,9.558347e+06,10.0,NaN,NaN,NaN,NaN
2.0,8.594413e+06,10.0,NaN,NaN,NaN,NaN
3.0,2.610991e+06,10.0,NaN,NaN,NaN,NaN
4.0,5.714483e+06,10.0,NaN,NaN,NaN,NaN
5.0,9.259973e+06,10.0,NaN,NaN,NaN,NaN
6.0,9.970838e+06,10.0,NaN,NaN,NaN,NaN
7.0,8.680822e+04,10.0,NaN,NaN,NaN,NaN
8.0,6.821470e+05,10.0,NaN,NaN,NaN,NaN
9.0,4.319340e+06,10.0,NaN,NaN,NaN,NaN
10.0,2.074685e+05,10.0,NaN,NaN,NaN,NaN


In [143]:
print_analyse(hist)

Analyse des donnees generees :



,Score,Nbr itrs,z1,z2,x3,x4
count,5.000000e+01,50.0,0.0,0.0,0.0,0.0
mean,3.219913e+08,10.0,NaN,NaN,NaN,NaN
std,2.236362e+09,0.0,NaN,NaN,NaN,NaN
min,8.680822e+04,10.0,NaN,NaN,NaN,NaN
25%,1.666651e+06,10.0,NaN,NaN,NaN,NaN
50%,4.367581e+06,10.0,NaN,NaN,NaN,NaN
75%,9.218233e+06,10.0,NaN,NaN,NaN,NaN
max,1.581915e+10,10.0,NaN,NaN,NaN,NaN


### Conclusion

En moyenne sur 50 exécutions des 3 algorithmes, l'algorithme aléatoire est celui qui a donné les meilleurs résultats ce qui est assez surprenant. Le deuxième meilleur résultat est celui du hill climbing avec le recuit simulé en dernière place avec ses trois variantes : refroidissement polynomial, exponentiel et logarithmique par ordre respectif. Mais cela ne reflète pas vraiment la performance générale de l'algorithme car d'un point je n'ai pas choisi un grand nombre d'itérations, et d'un autre point, les métaheuristiques du hill climbing et du recuit simulé ne sont pas parfaitement implémentes car plusieurs constantes doivent être précisées par expérimentation, ce que j'ai fait juste par intuition. Du coup, ces métaheuristiques ont encore besoin d'optimisation. J'ai choisi pour toutes les exécutions effectuées 10 itérations par cycle (50 cycles) et 0.01 pour epsilon. 
En ce qui concerne la durée d'exécution en termes de nombre d'itérations moyen des 3 algorithmes, il est le même. Normalement, on doit être capable de voir la différence dans le temps d'exécution en augmentant le nombre d'itérations.
En conclusion, le meilleur cout trouve est de 2025.4993228878886
